# 🏛️ Olympus Graph — Exploratory Data Analysis

This notebook explores the Kaggle **120 Years of Olympic History** dataset.

**Dataset:** `data/raw/athlete_events.csv`  
**Source:** https://www.kaggle.com/datasets/heesoo37/120-years-of-olympic-history

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style='whitegrid', palette='muted')
plt.rcParams['figure.figsize'] = (14, 6)

# Load data
df = pd.read_csv('../data/raw/athlete_events.csv')
print(f'Dataset shape: {df.shape}')
df.head()

## 1. Basic Statistics

In [ ]:
print(f'Years covered:    {df["Year"].min()} – {df["Year"].max()}')
print(f'Unique athletes:  {df["Name"].nunique():,}')
print(f'Unique events:    {df["Event"].nunique():,}')
print(f'Unique countries: {df["NOC"].nunique()}')
print(f'Total medals:     {df["Medal"].notna().sum():,}')
print()
df.describe()

In [ ]:
# Missing values
missing = df.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
print('Missing values:')
print(missing)

## 2. Medal Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Top 15 countries by total medals
top_countries = df[df['Medal'].notna()].groupby('NOC').size().nlargest(15)
top_countries.plot(kind='barh', ax=axes[0], color='#FFD700')
axes[0].set_title('Top 15 Countries by Total Medals')
axes[0].set_xlabel('Medal Count')

# Medal distribution by type
medal_counts = df['Medal'].value_counts()
colors = {'Gold': '#FFD700', 'Silver': '#C0C0C0', 'Bronze': '#CD7F32'}
medal_counts.plot(kind='bar', ax=axes[1], color=[colors[m] for m in medal_counts.index])
axes[1].set_title('Medal Distribution by Type')
axes[1].set_ylabel('Count')

plt.tight_layout()
plt.show()

## 3. Temporal Trends

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Athletes per year
athletes_per_year = df.groupby('Year')['Name'].nunique()
athletes_per_year.plot(ax=axes[0], marker='o', markersize=3)
axes[0].set_title('Number of Athletes per Olympic Games')
axes[0].set_ylabel('Athletes')

# Events per year
events_per_year = df.groupby('Year')['Event'].nunique()
events_per_year.plot(ax=axes[1], marker='o', markersize=3, color='orange')
axes[1].set_title('Number of Events per Olympic Games')
axes[1].set_ylabel('Events')

plt.tight_layout()
plt.show()

## 4. Athlete Age Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Age distribution
df['Age'].dropna().hist(bins=50, ax=axes[0], color='steelblue', edgecolor='white')
axes[0].set_title('Age Distribution of Olympic Athletes')
axes[0].set_xlabel('Age')

# Age of Gold medalists
gold = df[df['Medal'] == 'Gold']
gold['Age'].dropna().hist(bins=50, ax=axes[1], color='#FFD700', edgecolor='white')
axes[1].set_title('Age Distribution of Gold Medalists')
axes[1].set_xlabel('Age')

plt.tight_layout()
plt.show()

print(f'Mean age (all):     {df["Age"].mean():.1f}')
print(f'Mean age (gold):    {gold["Age"].mean():.1f}')

## 5. Sport / Event Analysis

In [ ]:
# Top sports by number of athletes
top_sports = df.groupby('Sport')['Name'].nunique().nlargest(20)

fig, ax = plt.subplots(figsize=(14, 8))
top_sports.plot(kind='barh', ax=ax, color='steelblue')
ax.set_title('Top 20 Sports by Number of Unique Athletes')
ax.set_xlabel('Unique Athletes')
plt.tight_layout()
plt.show()

## 6. Repeat Medalists (Model Feature Insight)

In [ ]:
# Athletes with most medals
medalists = df[df['Medal'].notna()]
top_medalists = medalists.groupby('Name').agg(
    total_medals=('Medal', 'count'),
    gold=('Medal', lambda x: (x == 'Gold').sum()),
    sports=('Sport', 'nunique'),
    years=('Year', lambda x: f"{x.min()}-{x.max()}"),
    country=('NOC', 'first'),
).nlargest(20, 'total_medals')

top_medalists

## 7. Feature Engineering Preview

These are the features we'll use for the GNN:

In [ ]:
# Compute "Momentum" — average medal performance over career
df['performance'] = df['Medal'].map({'Gold': 3, 'Silver': 2, 'Bronze': 1}).fillna(0)

career = df.groupby('Name').agg(
    avg_performance=('performance', 'mean'),
    num_games=('Year', 'nunique'),
    career_span=('Year', lambda x: x.max() - x.min()),
).query('num_games >= 2')

fig, ax = plt.subplots(figsize=(10, 6))
ax.scatter(career['num_games'], career['avg_performance'], alpha=0.3, s=5)
ax.set_xlabel('Number of Games Attended')
ax.set_ylabel('Average Performance Score')
ax.set_title('Performance vs. Experience (Multi-Games Athletes)')
plt.tight_layout()
plt.show()

## 8. Graph Statistics Preview

Node and edge counts for the knowledge graph:

In [ ]:
print('=== Projected Graph Statistics ===')
print(f'Athlete nodes:     {df["Name"].nunique():,}')
print(f'Country nodes:     {df["NOC"].nunique():,}')
print(f'Event nodes:       {df["Event"].nunique():,}')
print(f'Games nodes:       {df["Games"].nunique() if "Games" in df.columns else df.groupby(["Year", "Season"]).ngroups:,}')
print()
print(f'PARTICIPATED_IN:   {df.groupby(["Name", "Year"]).ngroups:,}')
print(f'REPRESENTS:        {df.groupby(["Name", "NOC"]).ngroups:,}')
print(f'COMPETED_IN:       {df.groupby(["Name", "Event", "Year"]).ngroups:,}')
print(f'WON_MEDAL:         {df[df["Medal"].notna()].groupby(["Name", "Event", "Year"]).ngroups:,}')